# Snow Depth and Snow Cover Data Exploration 

This tutorial demonstrates how to access and compare coincident snow data across in-situ, airborne, and satellite platforms from NASA's SnowEx, ASO, and MODIS data sets, respectively. All data are available from the NASA National Snow and Ice Data Center Distributed Active Archive Center, or NSIDC DAAC. 

### Here are the steps you will learn in this snow data notebook:

1. Explore the coverage and structure of select NSIDC DAAC snow data products, as well as available resources to search and access data.
2. Search and download spatiotemporally coincident data across in-situ, airborne, and satellite observations.
3. Subset and reformat MODIS data using the NSIDC DAAC API.
4. Read CSV and GeoTIFF formatted data using geopandas and rasterio libraries.
5. Subset data based on buffered area.
5. Extract and visualize raster values at point locations.
6. Save output as shapefile for further GIS analysis.


___

## Explore snow products and resources


### NSIDC introduction

[The National Snow and Ice Data Center](https://nsidc.org) provides over 1100 data sets covering the Earth's cryosphere and more, all of which are available to the public free of charge. Beyond providing these data, NSIDC creates tools for data access, supports data users, performs scientific research, and educates the public about the cryosphere. 

#### Select Data Resources

* [NSIDC Data Search](https://nsidc.org/data/search/#keywords=snow)
    * Search NSIDC snow data
* [NSIDC Data Update Announcements](https://nsidc.org/the-drift/data-update/) 
    * News and tips for data users
* [NASA Earthdata Search](http://search.earthdata.nasa.gov/)
    * Search and access data across the NASA Earthdata
* [NASA Worldview](https://worldview.earthdata.nasa.gov/)
    * Interactive interface for browsing full-resolution, global, daily satellite images
    
    
#### Snow Today

[Snow Today](https://nsidc.org/snow-today), a collaboration with the University of Colorado's Institute of Alpine and Arctic Research (INSTAAR), provides near-real-time snow analysis for the western United States and regular reports on conditions during the winter season. Snow Today is funded by NASA Hydrological Sciences Program and utilizes data from the Moderate Resolution Imaging Spectroradiometer (MODIS) instrument and snow station data from the Snow Telemetry (SNOTEL) network by the Natural Resources Conservation Service (NRCS), United States Department of Agriculture (USDA) and the California Department of Water Resources: www.wcc.nrcs.usda.gov/snow.

### Snow-related missions and data sets used in the following steps:

* [SnowEx](https://nsidc.org/data/snowex)
    * SnowEx17 Ground Penetrating Radar, Version 2: https://doi.org/10.5067/G21LGCNLFSC5
* [ASO](https://nsidc.org/data/aso)
    * ASO L4 Lidar Snow Depth 3m UTM Grid, Version 1: https://doi.org/10.5067/KIE9QNVG7HP0
* [MODIS](https://nsidc.org/data/modis)
    * MODIS/Terra Snow Cover Daily L3 Global 500m SIN Grid, Version 6: https://doi.org/10.5067/MODIS/MOD10A1.006


#### Other relevant snow products:

* [VIIRS Snow Data](http://nsidc.org/data/search/#sortKeys=score,,desc/facetFilters=%257B%2522facet_sensor%2522%253A%255B%2522Visible-Infrared%2520Imager-Radiometer%2520Suite%2520%257C%2520VIIRS%2522%255D%252C%2522facet_parameter%2522%253A%255B%2522SNOW%2520COVER%2522%252C%2522Snow%2520Cover%2522%255D%257D/pageNumber=1/itemsPerPage=25)

* [AMSR-E and AMSR-E/AMSR2 Unified Snow Data](http://nsidc.org/data/search/#sortKeys=score,,desc/facetFilters=%257B%2522facet_sensor%2522%253A%255B%2522Advanced%2520Microwave%2520Scanning%2520Radiometer-EOS%2520%257C%2520AMSR-E%2522%252C%2522Advanced%2520Microwave%2520Scanning%2520Radiometer%25202%2520%257C%2520AMSR2%2522%255D%252C%2522facet_parameter%2522%253A%255B%2522SNOW%2520WATER%2520EQUIVALENT%2522%252C%2522Snow%2520Water%2520Equivalent%2522%255D%257D/pageNumber=1/itemsPerPage=25)

* [MEaSUREs Snow Data](http://nsidc.org/data/search/#keywords=measures/sortKeys=score,,desc/facetFilters=%257B%2522facet_parameter%2522%253A%255B%2522SNOW%2520COVER%2522%255D%252C%2522facet_sponsored_program%2522%253A%255B%2522NASA%2520National%2520Snow%2520and%2520Ice%2520Data%2520Center%2520Distributed%2520Active%2520Archive%2520Center%2520%257C%2520NASA%2520NSIDC%2520DAAC%2522%255D%252C%2522facet_format%2522%253A%255B%2522NetCDF%2522%255D%252C%2522facet_temporal_duration%2522%253A%255B%252210%252B%2520years%2522%255D%257D/pageNumber=1/itemsPerPage=25)
  
* Near-Real-Time SSM/I-SSMIS EASE-Grid Daily Global Ice Concentration and Snow Extent (NISE), Version 5: https://doi.org/10.5067/3KB2JPLFPK3R

___
### Import Packages

Get started by importing packages needed to run the following code blocks, including the `tutorial_helper_functions` module provided within this repository.

In [ ]:
import os
import geopandas as gpd
from shapely.geometry import Polygon, mapping
from shapely.geometry.polygon import orient
import pandas as pd 
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import numpy as np
import pyresample as prs
import requests
import json
import pprint
from rasterio.mask import mask
from mpl_toolkits.axes_grid1 import make_axes_locatable


# This is our functions module. We created several helper functions to discover, access, and harmonize the data below.
import tutorial_helper_functions as fn

___


## Data Discovery

Start by identifying your study area and exploring coincident data over the same time and area. 

NASA Earthdata Search can be used to visualize file coverage over mulitple data sets and to access the same data you will be working with below: 
https://search.earthdata.nasa.gov/projects?projectId=5366449248


### Identify area and time of interest

Since our focus is on the Grand Mesa study site of the NASA SnowEx campaign, we'll use that area to search for coincident data across other data products. From the [SnowEx17 Ground Penetrating Radar Version 2](https://doi.org/10.5067/G21LGCNLFSC5) landing page, you can find the rectangular spatial coverage under the Overview tab, or you can draw a polygon over your area of interest in the map under the Download Data tab and export the shape as a geojson file using the Export Polygon icon shown below. An example polygon geojson file is provided in the /Data folder of this repository.   

<img align="left" src="Data-download-polygon-export.png">


#### Create polygon coordinate string

Read in the geojson file as a GeoDataFrame object and simplify and reorder using the shapely package. This will be converted back to a dictionary to be applied as our polygon search parameter. 

In [ ]:
polygon_filepath = str(os.getcwd() + '/Data/nsidc-polygon.json') # Note: A shapefile or other vector-based spatial data format could be substituted here.

gdf = gpd.read_file(polygon_filepath) #Return a GeoDataFrame object

# Simplify polygon for complex shapes in order to pass a reasonable request length to CMR. The larger the tolerance value, the more simplified the polygon.
# Orient counter-clockwise: CMR polygon points need to be provided in counter-clockwise order. The last point should match the first point to close the polygon.
poly = orient(gdf.simplify(0.05, preserve_topology=False).loc[0],sign=1.0)

#Format dictionary to polygon coordinate pairs for CMR polygon filtering
polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])
print('Polygon coordinates to be used in search:', polygon)
poly

#### Set time range

We are interested in accessing files within each data set over the same time range, so we'll start by searching all of 2017.

In [ ]:
temporal = '2017-01-01T00:00:00Z,2017-12-31T23:59:59Z' # Set temporal range

### Create data dictionary 

Create a nested dictionary with each data set shortname and version, as well as shared temporal range and polygonal area of interest. Data set shortnames, or IDs, as well as version numbers, are located at the top of every NSIDC landing page.

In [ ]:
data_dict = { 'snowex': {'short_name': 'SNEX17_GPR','version': '2','polygon': polygon,'temporal':temporal},
                'aso': {'short_name': 'ASO_3M_SD','version': '1','polygon': polygon,'temporal':temporal},
             'modis': {'short_name': 'MOD10A1','version': '6','polygon': polygon,'temporal':temporal}
            }

### Determine how many files exist over this time and area of interest, as well as the average size and total volume of those files

We will use the `granule_info` function to query metadata about each data set and associated files using the [Common Metadata Repository (CMR)](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html), which is a high-performance, high-quality, continuously evolving metadata system that catalogs Earth Science data and associated service metadata records. Note that not all NSIDC data can be searched at the file level using CMR, particularly those outside of the NASA DAAC program. 

In [ ]:
for k, v in data_dict.items(): fn.granule_info(data_dict[k])

### Find coincident data

The function above tells us the size of data available for each data set over our time and area of interest, but we want to go a step further and determine what time ranges are coincident based on our bounding box. This `time_overlap` helper function returns a dataframe with file names, dataset_id, start date, and end date for all files that overlap in temporal range across all data sets of interest. 

In [ ]:
search_df = fn.time_overlap(data_dict)
print(len(search_df), ' total files returned')
search_df

___

## Data Access

The number of files has been greatly reduced to only those needed to compare data across these data sets. This CMR query will collect the data file URLs, including the associated quality and metadata files if available.

In [ ]:
# Create new dictionary with fields needed for CMR url search

url_df = search_df.drop(columns=['start_date', 'end_date','version','dataset_id'])
url_dict = url_df.to_dict('records')

# CMR search variables
granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
headers= {'Accept': 'application/json'}

# Create URL list from each df row
urls = []
for i in range(len(url_dict)):
    response = requests.get(granule_search_url, params=url_dict[i], headers=headers)
    results = json.loads(response.content)
    urls.append(fn.cmr_filter_urls(results))
# flatten url list
urls = list(np.concatenate(urls))
urls

### Additional data access and subsetting services

#### API Access
Data can be accessed directly from our HTTPS file system through the URLs collected above, or through our Application Programming Interface (API). Our API offers you the ability to order data using specific temporal and spatial filters, as well as subset, reformat, and reproject select data sets. The same subsetting, reformatting, and reprojection services available on select data sets through NASA Earthdata Search can also be applied using this API. These options can be requested in a single access command without the need to script against our data directory structure. See our [programmatic access guide](https://nsidc.org/support/how/how-do-i-programmatically-request-data-services) for more information on API options. 

#### Add service request options for MODIS data

According to https://nsidc.org/support/faq/what-data-subsetting-reformatting-and-reprojection-services-are-available-for-MODIS-data, we can see that spatial subsetting and GeoTIFF reformatting are available for MOD10A1 so those options are requested below. The area subset must be described as a bounding box, which can be created based on the polygon bounds above. We will also add GeoTIFF reformatting to the MOD10A1 data dictionary and the temporal range will be set based on the range of MOD10A1 files in the dataframe above. These new parameters will be added to the API request below.

In [ ]:
bounds = poly.bounds # Get polygon bounds to be used as bounding box input
data_dict['modis']['bbox'] = ','.join(map(str, list(bounds))) # Add bounding box subsetting to MODIS dictionary
data_dict['modis']['format'] = 'GeoTIFF' # Add geotiff reformatting to MODIS dictionary

# Set new temporal range based on dataframe above. Note that this will request all MOD10A1 data falling within this time range.
modis_start = min(search_df.loc[search_df['short_name'] == 'MOD10A1', 'start_date'])
modis_end = max(search_df.loc[search_df['short_name'] == 'MOD10A1', 'end_date'])
data_dict['modis']['temporal'] = ','.join([modis_start,modis_end])
print(data_dict['modis'])

#### Create the data request API endpoint
Programmatic API requests are formatted as HTTPS URLs that contain key-value-pairs specifying the service operations that we specified above. We will first create a string of key-value-pairs from our data dictionary and we'll feed those into our API endpoint. This API endpoint can be executed via command line, a web browser, or in Python below.

In [ ]:
base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request' # Set NSIDC data access base URL
#data_dict['modis']['request_mode'] = 'stream' # Set the request mode to asynchronous

param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in data_dict['modis'].items()) # Convert param_dict to string
param_string = param_string.replace("'","") # Remove quotes

api_request = [f'{base_url}?{param_string}']
print(api_request[0]) # Print API base URL + request parameters

### Download options

The following functions will return the file URLs and the MOD10A1 API request. For demonstration purposes, these functions have been commented out, and instead the data utilized in the following steps will be accessed from a staged directory. ***Note that if you are running this notebook in Binder, the memory may not be sufficient to download these files. Please use the Docker or local Conda options provided in the README if you are interested in downloading all files.***

In [ ]:
from pathlib import Path

path = Path(".").absolute() / "Data"

if not os.path.exists(path):
     os.mkdir(path)

print(f"Downloading data from S3 to {path}")
os.chdir(path)
# pull data from staged bucket for demonstration
!awscliv2 --no-sign-request s3 cp s3://snowex-aso-modis-tutorial-data/ ./ --recursive #access data in staged directory

___
## Read in SnowEx data and buffer points around Snotel location

This SnowEx data set is provided in CSV. A [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html) is used to easily read in data. For these next steps, just one day's worth of data will be selected from this file and the coincident ASO and MODIS data will be selected.


In [ ]:
snowex_path = './SnowEx17_GPR_Version2_Week1.csv' # Define local filepath
df = pd.read_csv(snowex_path, sep='\t') 
df.head()

### Convert to time values and extract a single day

The collection date needs to be extracted from the `collection` value and a new dataframe will be generated as a subset of the original based on a single day:

In [ ]:
df['date'] = df.collection.str.rsplit('_').str[-1].astype(str)
df.date = pd.to_datetime(df.date, format="%m%d%y")
df = df.sort_values(['date'])
df_subset = df[df['date'] == '2017-02-08'] # subset original dataframe and only select this date
df.head()

#### Convert to Geopandas dataframe to provide point geometry

According to the SnowEx documentation, the data are available in UTM Zone 12N so we'll set to this projection so that we can buffer in meters in the next step:

In [ ]:
gdf_utm= gpd.GeoDataFrame(df_subset, geometry=gpd.points_from_xy(df_subset.x, df_subset.y), crs='EPSG:32612')
gdf_utm.head()

### Buffer data around SNOTEL site

We can further subset the SnowEx snow depth data to get within a 500 m radius of the [SNOTEL Mesa Lakes](https://wcc.sc.egov.usda.gov/nwcc/site?sitenum=622&state=co) site.

First we'll create a new geodataframe with the SNOTEL site location, set to our SnowEx UTM coordinate reference system, and create a 500 meter buffer around this point. Then we'll subset the SnowEx points to the buffer and convert back to the WGS84 CRS:

In [ ]:
# Create another geodataframe (gdfsel) with the center point for the selection
df_snotel = pd.DataFrame(
    {'SNOTEL Site': ['Mesa Lakes'],
     'Latitude': [39.05],
     'Longitude': [-108.067]})
gdf_snotel = gpd.GeoDataFrame(df_snotel, geometry=gpd.points_from_xy(df_snotel.Longitude, df_snotel.Latitude), crs='EPSG:4326')

gdf_snotel.to_crs('EPSG:32612', inplace=True) # set CRS to UTM 12 N

buffer = gdf_snotel.buffer(500) #create 500 m buffer

gdf_buffer = gdf_utm.loc[gdf_utm.geometry.within(buffer.unary_union)] # subset dataframe to buffer region
gdf_buffer = gdf_buffer.to_crs('EPSG:4326')

___
## Read in Airborne Snow Observatory data and clip to SNOTEL buffer

Snow depth data from the ASO L4 Lidar Snow Depth 3m UTM Grid data set were calculated from surface elevation measured by the Riegl LMS-Q1560 airborne laser scanner (ALS). The data are provided in GeoTIFF format, so we'll use the [Rasterio](https://rasterio.readthedocs.io/en/latest/) library to read in the data. 

In [ ]:
aso_path = './ASO_3M_SD_USCOGM_20170208.tif' # Define local filepath

aso = rasterio.open(aso_path)

### Clip data to SNOTEL buffer

In order to reduce the data volume to the buffered region of interest, we can subset this GeoTIFF to the same SNOTEL buffer:

In [ ]:
buffer = buffer.to_crs(crs=aso.crs) # convert buffer to CRS of ASO rasterio object
out_img, out_transform = mask(aso, buffer, crop=True)
out_meta = aso.meta.copy()
epsg_code = int(aso.crs.data['init'][5:])
out_meta.update({"driver": "GTiff", "height": out_img.shape[1], "width": out_img.shape[2], "transform": out_transform, "crs": '+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs'})
out_tif = 'clipped_ASO_3M_SD_USCOGM_20170208.tif'

with rasterio.open(out_tif, 'w', **out_meta) as dest:
    dest.write(out_img)
    
clipped_aso = rasterio.open(out_tif)
aso_array = clipped_aso.read(1, masked=True)

___ 
## Read in MODIS Snow Cover data 

We are interested in the Normalized Difference Snow Index (NDSI) snow cover value from the MOD10A1 data set, which is an index that is related to the presence of snow in a pixel. According to the [MOD10A1 FAQ](https://nsidc.org/support/faq/what-ndsi-snow-cover-and-how-does-it-compare-fsc), snow cover is detected using the NDSI ratio of the difference in visible reflectance (VIS) and shortwave infrared reflectance (SWIR), where NDSI = ((band 4-band 6) / (band 4 + band 6)).

Note that you may need to change this filename output below if you download the data outside of the staged bucket, as the output names may vary per request. 

In [ ]:
modis_path = './MOD10A1_A2017039_h09v05_006_2017041102600_MOD_Grid_Snow_500m_NDSI_Snow_Cover_99f6ee91_subsetted.tif' # Define local filepath
modis = rasterio.open(modis_path)
modis_array = modis.read(1, masked=True)

___
## Add ASO and MODIS data to GeoPandas dataframe

In order to add data from these ASO and MODIS gridded data sets, we need to define the geometry parameters for theses, as well as the SnowEx data. The SnowEx geometry is set using the longitude and latitude values of the geodataframe:

In [ ]:
snowex_geometry = prs.geometry.SwathDefinition(lons=gdf_buffer['long'], lats=gdf_buffer['lat'])
print('snowex geometry: ', snowex_geometry)

With ASO and MODIS data on regular grids, we can create area definitions for these using projection and extent metadata:

In [ ]:
pprint.pprint(clipped_aso.profile)
print('')
print(clipped_aso.bounds)


pprint.pprint(modis.profile)
print('')
print(modis.bounds)

In [ ]:
# Create area definition for ASO
area_id = 'UTM_13N' # area_id: ID of area
description = 'WGS 84 / UTM zone 13N' # description: Description
proj_id = 'UTM_13N' # proj_id: ID of projection (being deprecated)
projection = 'EPSG:32613' # projection: Proj4 parameters as a dict or string
width = clipped_aso.width # width: Number of grid columns
height = clipped_aso.height # height: Number of grid rows
area_extent = (234081.0, 4326303.0, 235086.0, 4327305.0)
aso_geometry = prs.geometry.AreaDefinition(area_id, description, proj_id, projection, width, height, area_extent)

# Create area definition for MODIS
area_id = 'Sinusoidal' # area_id: ID of area
description = 'Sinusoidal Modis Spheroid' # description: Description
proj_id = 'Sinusoidal' # proj_id: ID of projection (being deprecated)
projection = 'PROJCS["Sinusoidal Modis Spheroid",GEOGCS["Unknown datum based upon the Authalic Sphere",DATUM["Not_specified_based_on_Authalic_Sphere",SPHEROID["Sphere",6371007.181,887203.3395236016,AUTHORITY["EPSG","7035"]],AUTHORITY["EPSG","6035"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4035"]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]' # projection: Proj4 parameters as a dict or string
width = modis.width # width: Number of grid columns
height = modis.height # height: Number of grid rows
area_extent = (-9332971.361735353, 4341240.1538655795, -9331118.110869242, 4343093.404731691)
modis_geometry = prs.geometry.AreaDefinition(area_id, description, proj_id, projection, width, height, area_extent)

### Interpolate ASO and MODIS values onto SnowEx points

To interpolate ASO snow depth and MODIS snow cover data to SnowEx snow depth points, we can use the `pyresample` library. The `radius_of_influence` parameter determines maximum radius to look for nearest neighbor interpolation.

In [ ]:
# add ASO values to geodataframe
import warnings
warnings.filterwarnings('ignore') # ignore warning when resampling to a different projection
gdf_buffer['aso_snow_depth'] = prs.kd_tree.resample_nearest(aso_geometry, aso_array, snowex_geometry, radius_of_influence=3)

# add MODIS values to geodataframe
gdf_buffer['modis_ndsi'] = prs.kd_tree.resample_nearest(modis_geometry, modis_array, snowex_geometry, radius_of_influence=500)

gdf_buffer.head()

___ 
## Visualize data and export for further GIS analysis

The rasterio plot module allows you to directly plot GeoTIFFs objects. The SnowEx `Thickness` values are plotted against the clipped ASO snow depth raster.

In [ ]:
gdf_buffer_aso_crs = gdf_buffer.to_crs('EPSG:32613') 

from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(figsize=(10, 10))
show(clipped_aso, ax=ax)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1) # fit legend to height of plot
gdf_buffer_aso_crs.plot(column='Thickness', ax=ax, cmap='OrRd', legend=True, cax=cax, legend_kwds=
                        {'label': "Snow Depth (m)",});

We can do the same for MOD10A1: This was subsetted to the entire Grand Mesa region defined by the SnowEx data set coverage. 

In [ ]:
# Set dataframe to MOD10A1 Sinusoidal projection
gdf_buffer_modis_crs = gdf_buffer.to_crs('PROJCS["Sinusoidal Modis Spheroid",GEOGCS["Unknown datum based upon the Authalic Sphere",DATUM["Not_specified_based_on_Authalic_Sphere",SPHEROID["Sphere",6371007.181,887203.3395236016,AUTHORITY["EPSG","7035"]],AUTHORITY["EPSG","6035"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4035"]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]')

fig, ax = plt.subplots(figsize=(10, 10))
show(modis, ax=ax)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1) # fit legend to height of plot
gdf_buffer_modis_crs.plot(column='Thickness', ax=ax, cmap='OrRd', legend=True, cax=cax, legend_kwds=
                        {'label': "Snow Depth (m)",});

### Additional data imagery services

#### NASA Worldview and the Global Browse Imagery Service

NASA’s EOSDIS Worldview mapping application provides the capability to interactively browse over 900 global, full-resolution satellite imagery layers and then download the underlying data. Many of the available imagery layers are updated within three hours of observation, essentially showing the entire Earth as it looks “right now."

According to the [MOD10A1 landing page](https://nsidc.org/data/mod10a1), snow cover imagery layers from this data set are available through NASA Worldview. This layer can be downloaded as various image files including GeoTIFF using the snapshot feature at the top right of the page. This link presents the MOD10A1 NDSI layer over our time and area of interest: https://go.nasa.gov/35CgYMd. 

Additionally, the NASA Global Browse Imagery Service provides up to date, full resolution imagery for select NSIDC DAAC data sets as web services including WMTS, WMS, KML, and more. These layers can be accessed in GIS applications following guidance on the [GIBS documentation pages](https://wiki.earthdata.nasa.gov/display/GIBS/Geographic+Information+System+%28GIS%29+Usage). 

### Export dataframe to Shapefile

Finally, the dataframe can be exported as an Esri shapefile for further analysis in GIS:

In [ ]:
gdf_buffer = gdf_buffer.drop(columns=['date'])
gdf_buffer.to_file('snow-data-20170208.shp')